![HSV-AI Logo](https://github.com/HSV-AI/hugo-website/blob/master/static/images/logo_v9.png?raw=true)

# Implicit Recommendation from ECommerce Data

Some of the material for this work is based on [A Gentle Introduction to Recommender Systems with Implicit Feedback](https://jessesw.com/Rec-System/) by Jesse Steinweg Woods. This tutorial includes an implementation of the Alternating Least Squares algorithm and some other useful functions (like the area under the curve calculation). Other parts of the tutorial are based on a previous version of the Implicit library and had to be reworked.

The dataset used for this work is from Kaggle [Vipin Kumar Transaction Data](https://www.kaggle.com/vipin20/transaction-data):

## Context

This is a item purchased transactions data. It has 8 columns.
This data makes you familer with transactions data.

## Content

Data description is :-

* UserId -It is a unique ID for all User Id
* TransactionId -It contains unique Transactions ID
* TransactionTime -It contains Transaction Time
* ItemCode -It contains item code that item will be purchased
* ItemDescription -It contains Item description
* NumberOfItemPurchased -It contains total number of items Purchased
* CostPerltem -Cost per item Purchased
* Country -Country where item purchased

## Basics of EDA

Here are a few things that we are looking for in the invoice / transaction data:

1. Were there any negative totals? If so why?
2. What percentage of the purchases actually contained multiple items?
3. What is the spread of purchases by customer ID? Do we have a few customers whose behavior may drive recommendations in a way that doesn't fit the average customer?
4. Where there any purchases that were VERY large? If so why? Do we want to include these values to train model behavior?
5. Is there any missing data that we need to scrub?


In [ ]:
%reload_kedro

In [ ]:
import pandas as pd
import numpy as np
import random
import scipy.sparse
from matplotlib import pyplot as plt
from pandas.api.types import CategoricalDtype


# Available Files

Let's go ahead and look into some of these files and see what we can see.

In [ ]:
transactions = catalog.load("vipin20_kaggle_data")

transactions.head()

# Checking for missing data

In [ ]:
print('Total length is',len(transactions))
transactions.isna().sum()

In [ ]:
transaction_counts = transactions['TransactionId'].value_counts().to_numpy()
print('There are', len(transaction_counts), 'unique transactions\n')
print('Here are the counts of transactions ordered from largest to smallest')
print(transaction_counts)
print('\nAnd a graph of what the curve looks like:')
plt.plot(transaction_counts) 
plt.show()

# User Interactions

Let's take a look at how many unique customers are included in this dataset

In [ ]:
user_counts = transactions['UserId'].value_counts().to_numpy()
print('There are', len(user_counts), 'unique customers\n')
print('Here are the counts of transactions per customer ordered from largest to smallest')
print(user_counts)
print('\nAnd a graph of what the curve looks like:')
plt.plot(user_counts) 
plt.show()

In [ ]:
transactions.groupby(['UserId'])['UserId'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)

It looks like "-1" is used when the customer is unknown. Let's take a look at the UserId with the very high count of items in the transactions.

In [ ]:
transactions[transactions.UserId == 374661].groupby(transactions.TransactionId).count()

It appears that there are a lot of different transactions, so probably not just the same thing being purchased over and over. Not really sure what to do with this at the moment.

# Transactions over Time

Now we need to look at the number of items purchased each day to see if there is anything interesting that pops out.

In [ ]:
from datetime import datetime

datetime_object = datetime.strptime('Mon Feb 12 04:26:00 IST 2018', '%a %b %d %H:%M:%S IST %Y')

def func(date):
    temp = datetime.strptime(str(date), '%a %b %d %H:%M:%S IST %Y')
    return temp.strftime('%Y%m%d')

transactions['datetime'] = transactions.apply(lambda x: func(x.TransactionTime), axis=1)


In [ ]:
transactions.groupby(['datetime'])['datetime'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['datetime'], ascending=True) \
                             .plot(figsize=(15,10))

# Checking Invoice Totals

We need to make sure all the invoice totals that we're using are positive - this keeps us from using invoices that captured customer returned items.

In [ ]:
transactions['StockTotal'] = transactions['NumberOfItemsPurchased'] * transactions['CostPerItem']
totals = transactions.groupby(transactions.TransactionId)['StockTotal'].sum()
totals.plot()

In [ ]:
totals.sort_values(ascending=False)

In [ ]:
print('There are', len(transactions[transactions.NumberOfItemsPurchased < 0]), 'negative quantities')
transactions[transactions.NumberOfItemsPurchased < 0].head()

It looks like the negative quantities are driven by returned items. Let's see if anyone purchased and returned items in the same transaction:

In [ ]:
temp_df = transactions.groupby(transactions.TransactionId).agg(minQ=('NumberOfItemsPurchased', 'min'), 
                               maxQ=('NumberOfItemsPurchased', 'max'))
temp_df[(temp_df.minQ < 0) & (temp_df.maxQ > 0)].head()

In [ ]:
print('There are', len(transactions[transactions.CostPerItem < 0]), 'negative prices')
transactions[transactions.CostPerItem < 0].head()

We can also check these transactions to see if there was a mix of negative prices along with positive.

In [ ]:
temp_df = transactions.groupby(transactions.TransactionId).agg(minQ=('CostPerItem', 'min'), 
                               maxQ=('CostPerItem', 'max'))
temp_df[(temp_df.minQ < 0) & (temp_df.maxQ > 0)].head()

Now that we have identified the cause of negative totals, we can remove them so that they do not affect our model.

In [ ]:
transactions = transactions[(transactions.NumberOfItemsPurchased > 0) & (transactions.CostPerItem > 0)]

Now we can check the totals that were much higher than average

In [ ]:
transactions[transactions.TransactionId == 5900323].head()

In [ ]:
transactions[transactions.TransactionId == 6396313].head()

It looks like there are two distinct cases where we may have an issue:

1. Extremely high priced items
2. Extremely high numbers of items purchased on the same transaction

We can remove both of these by just keeping the values within the 98% quantile

In [ ]:
q = transactions["CostPerItem"].quantile(0.98)
transactions = transactions[transactions["CostPerItem"] < q]

In [ ]:
q = transactions["NumberOfItemsPurchased"].quantile(0.98)
transactions = transactions[transactions["NumberOfItemsPurchased"] < q]

We will need to remove transactions that only included a single item

In [ ]:
minimum_order_size = 2
order_group = transactions.loc[:, ['TransactionId', 'ItemCode']].groupby('TransactionId').count()
 
multi_order = order_group[(order_group.ItemCode >= minimum_order_size)].count()
single_order = order_group[(order_group.ItemCode < minimum_order_size)].count()
 
print('Orders with at least',minimum_order_size,'products:',multi_order['ItemCode'])
print('Orders with less than',minimum_order_size,'products:',single_order['ItemCode'])
 
# We can capture the list of mutiple product orders with this:
order_filter = order_group[(order_group.ItemCode >= minimum_order_size)].index.tolist()

Well it looks like this entire dataset has transactions with multiple products. No need to filter out transactions with only a single.

# Data Sparcity

Let's take a look at the sparcity of the data. This will tell us how many products were purchased across multiple orders. This is directly related to how well a recommendation system can be trained.

In [ ]:
transaction_list = list(np.sort(transactions.TransactionId.unique())) # Get our unique customers
item_list = list(transactions.ItemCode.unique()) # Get our unique products that were purchased
quantity_list = list(transactions.NumberOfItemsPurchased) # All of our purchases

cols = transactions.TransactionId.astype(CategoricalDtype(categories=transaction_list, ordered=True)).cat.codes 
# Get the associated row indices
rows = transactions.ItemCode.astype(CategoricalDtype(categories=item_list, ordered=True)).cat.codes 
# Get the associated column indices
purchases_sparse = scipy.sparse.csr_matrix((quantity_list, (rows, cols)), shape=(len(item_list), len(transaction_list)))

In [ ]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

In [ ]:
plt.figure(figsize=(15, 15))
plt.spy(purchases_sparse, markersize=1, aspect='auto')

# Storing Interim Data

Now that we have the data cleaned up a bit and formatted correctly, we can save it to an interim file to be picked up by the model training algorithm.

In [ ]:
renamed_df = transactions.rename(columns={"TransactionId": "order_id", 
                            "ItemCode": "product_id", 
                            "ItemDescription":"description",
                            "NumberOfItemsPurchased":"quantity",
                            "CostPerItem":"price"})[['order_id', 'product_id', 'description', 'quantity']]

catalog.save("vipin20_transactions", renamed_df)

item_lookup = transactions[['ItemCode', 'ItemDescription']].drop_duplicates() # Only get unique item/description pairs
item_lookup['ItemCode'] = item_lookup.ItemCode.astype(str) # Encode as strings for future lookup ease

products_df = item_lookup.rename(columns={"ItemCode":"product_id", "ItemDescription":"description"})
catalog.save("vipin20_products", products_df)